In [1]:
import json
from datasets import load_dataset

/home/lamdo/miniconda3/envs/venv310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("/scratch/lamdo/precompute_keyphrase_extraction/scidocs_queries--retrieval_based_ukg_custom_trained_combined_references_nounphrase_v6-1_position_penalty+length_penalty.json") as f:
    eru_kg_generations = json.load(f)

with open("/scratch/lamdo/precompute_keyphrase_extraction/scidocs_queries--uokg-1.json") as f:
    uokg_generations = json.load(f)

with open("/scratch/lamdo/precompute_keyphrase_extraction/scidocs_queries--autokeygen-1.json") as f:
    autokeygen_generations = json.load(f)

with open("/scratch/lamdo/precompute_keyphrase_extraction/scidocs_queries--copyrnn-1.json") as f:
    copyrnn_generations = json.load(f)

In [3]:
ds = load_dataset("BeIR/scidocs", "queries")

In [4]:
INDEX = 40
ds["queries"][INDEX]

{'_id': 'df0ef17f63582603dafb1ac5c489dec1416ecbf4',
 'title': '',
 'text': 'Real World BCI: Cross-Domain Learning and Practical Applications'}

In [5]:
def check_truly_absent(phrase, text):
    text_tokens = set(text.split())
    tokens = phrase.split()
    return not all([tok in text for tok in tokens])

In [7]:
model_names = ["eru_kg", "autokeygen", "uokg", "copyrnn"]

for i, generations in enumerate([eru_kg_generations, autokeygen_generations, uokg_generations, copyrnn_generations]):
    keyphrases = generations[INDEX].get("automatically_extracted_keyphrases")
    present_keyphrases = keyphrases.get("present_keyphrases")[:10]
    absent_keyphrases = keyphrases.get("absent_keyphrases")[:10]

    truly_absent_keyphrases = [item for item in absent_keyphrases if check_truly_absent(item, ds["queries"][INDEX]["text"].lower())]

    print(model_names[i])
    print("present:", ", ".join(present_keyphrases))
    print("absent:",", ".join(absent_keyphrases))

    print(len(truly_absent_keyphrases) / len(absent_keyphrases))
    print("\n")

eru_kg
present: real world bci, cross-domain learning, practical applications, domain, applications
absent: target domain, source domain, domain adaptation, cross domain, deep learning, domain shift, transfer learning, cross domain recommendation, brain computer interface, meta learning
0.9


autokeygen
present: practical applications, cross-domain learning, real world, real world bci, bci, practical application, learning, world bci, applications, practical
absent: practical learning, real world applications, learning applications, bci applications, learning models, learning system, learning method, learning model, learning methods, practical systems
0.6


uokg
present: real world bci, real world, world bci, practical applications, cross-domain learning, world, real, bci, applications, learning
absent: real world applications, bci applications, practical learning, learning applications, real applications, real world practical applications, practical real world applications, practical l